In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

In [2]:
from utils.fourier import apply_fft_df, apply_fft_df_batch
import os

# Introduction

This notebook improves the algorithm for movement detection, developed in fourier_standstill.ipynb. We will use two threshold to detect hand flapping movement:
1. The maximum value of Fourier transform amplitude of standing still data
2. The maximum distance between acceleration magnitude values and baseline acceleration magnitude (1g).

# Calculate the threshold for acceleration magnitude

Below, we process the standstill data to how much the acceleration magnitude would deviate from the baseline value (1g), even if we stand still. We will use this value as threshold to detect movement.

In [3]:
# Get threshold values for acceleration magnitude

# Import data
import_folder_path = "./data/raw_data/"
keyword = "standstill"
csv_files = [f for f in os.listdir(import_folder_path) if 'csv' in f and keyword in f]

# Iterate through csv files to find the min and max values of acceleration magnitude
min_accel_mag = 1
max_accel_mag = 1
for fname in csv_files:
    df = pd.read_csv('./data/raw_data/' + fname, usecols=['timestamp', 'accel.x', 'accel.y', 'accel.z'])
    # Preprocess data
    df = df.iloc[500:-500].reset_index(drop=True) # Drop the first and last 500 rows (10 seconds) to remove noises caused by button pressings
    df['accel.mag'] = np.sqrt(df['accel.x']**2 + df['accel.y']**2 + df['accel.z']**2) # Create a new column for acceleration magnitude
    print("File: ", fname)
    print("Min value: ", df['accel.mag'].min())
    print("Max value: ", df['accel.mag'].max())
    min_accel_mag = min(min_accel_mag, df['accel.mag'].min())
    max_accel_mag = max(max_accel_mag, df['accel.mag'].max())
    print("-" * 50)

print("Final min value: ", min_accel_mag)
print("Final max value: ", max_accel_mag)
print("-" * 50)


accel_mag_threshold = max(np.absolute(max_accel_mag - 1), np.absolute(max_accel_mag - 1))
print("Final threshold values: ", accel_mag_threshold)

File:  standstill-20240919-16h50.csv
Min value:  0.9986257274107763
Max value:  1.038403617640402
--------------------------------------------------
File:  standstill-20240921-15h41.csv
Min value:  1.0043518695220417
Max value:  1.0347167467502534
--------------------------------------------------
Final min value:  0.9986257274107763
Final max value:  1.038403617640402
--------------------------------------------------
Final threshold values:  0.0384036176404019


# Import the threshold for Fourier transform amplitude

Import the Fourier transform table of standstill data, to identify the threshold

In [4]:
df_baseline = pd.read_csv('./data/fourier_transform/standstill-fft-results.csv')
df_baseline

,Frequency_Hz,mean_amp,std_amp,min_amp,max_amp
0,0.0,50.929353,0.122308,50.470849,51.295599
1,1.0,0.075142,0.039684,0.005109,0.296233
2,2.0,0.050548,0.028064,0.000252,0.178287
3,3.0,0.032716,0.019494,0.001493,0.161021
4,4.0,0.022261,0.012380,0.000092,0.097713
5,5.0,0.014932,0.008648,0.000220,0.068999
6,6.0,0.009307,0.006039,0.000182,0.048356
7,7.0,0.009167,0.005305,0.000108,0.047355
8,8.0,0.010316,0.005782,0.000233,0.047606
9,9.0,0.010789,0.006342,0.000188,0.041395


In [5]:
# Find the max amplitude of 1-24 Hz bands
max(df_baseline[-24:]['max_amp'])

0.2962334067485641

# Detect Handflapping

In [6]:
# Set variables to import data
import_folder_path = "./data/raw_data/"
keyword = "handflapping"
csv_files = [f for f in os.listdir(import_folder_path) if 'csv' in f and keyword in f]

In [7]:
# Initialize lists to store results
processed_df = [] # Store processed dataframes
results = [] # Store results

In [8]:
for fname in csv_files:
    # Import data
    df = pd.read_csv('./data/raw_data/' + fname, usecols=['timestamp', 'accel.x', 'accel.y', 'accel.z'])

    # Preprocess data
    df = df.iloc[500:-500].reset_index(drop=True) # Drop the first and last 500 rows (10 seconds) to remove noises caused by button pressings
    df['accel.mag'] = np.sqrt(df['accel.x']**2 + df['accel.y']**2 + df['accel.z']**2) # Create a new column for acceleration magnitude
    df['file_name'] = fname
    df['detected'] = np.repeat(False, len(df)) # Create a new column to store detection results

    # Calculate FFT
    handflapping_fft_results = apply_fft_df(df=df, file_name=fname, window_duration=1, sampling_rate=50, plot=False, column='accel.mag')

    # Set variables for detection
    n_windows = max(handflapping_fft_results['Window']) # Number of segments
    baseline_amplitudes = max(df_baseline[-24:]['max_amp']) # Max amplitude of 1-24 Hz bands of standstill data

    # Iterate through segements to detect handflapping 
    for i in range(n_windows + 1):
        # Store the Fourier transform results of the current window, i.e., segement
        window_results = handflapping_fft_results[handflapping_fft_results['Window'] == i]
        window = df[df['Window'] == i]
        window_amplitudes = (window_results[-24:]['Amplitude']).to_numpy()

        # Check if the amplitude of any frequency in the window is greater than the baseline amplitude
        if any(window_amplitudes > baseline_amplitudes):
            # Check if the maximum deviation of acceleration magnitude is greater than the threshold value
            accel_shift = np.max(np.absolute(df.loc[df['Window'] == i, 'accel.mag'] - 1))
            if accel_shift > accel_mag_threshold:
                df.loc[df['Window'] == i, 'detected'] = True
            else:
                continue
        else:
            continue


    # Drop rows with NaN values in the 'Window' column, i.e., rows that are not part of any window / segment
    df = df.dropna(subset=['Window']).reset_index(drop=True)

    # Append the processed data to the list
    processed_df.append(df)
    results.append(handflapping_fft_results)

In [9]:
# Concatenate & store all processed data
df = pd.concat(processed_df, ignore_index=True)
df.to_csv('./data/fourier_transform/handflapping-processed-detected-data.csv', index=False)
df_results = pd.concat(results, ignore_index=True)
df_results.to_csv('./data/fourier_transform/handflapping-fft-results.csv', index=False)

In [10]:
df.head()

,timestamp,accel.x,accel.y,accel.z,accel.mag,file_name,detected,Window
0,1.726238e+12,-0.922607,0.633545,0.143066,1.128296,handflapping-20240913-16h32.csv,True,0.0
1,1.726238e+12,-0.966431,0.498169,0.109497,1.092772,handflapping-20240913-16h32.csv,True,0.0
2,1.726238e+12,-0.966431,0.498169,0.109497,1.092772,handflapping-20240913-16h32.csv,True,0.0
3,1.726238e+12,-0.966431,0.498169,0.109497,1.092772,handflapping-20240913-16h32.csv,True,0.0
4,1.726238e+12,-0.966431,0.498169,0.109497,1.092772,handflapping-20240913-16h32.csv,True,0.0


In [11]:
df_results.head()

,Frequency_Hz,Amplitude,Window,file_name
0,0.0,51.388647,0,handflapping-20240913-16h32.csv
1,1.0,0.345220,0,handflapping-20240913-16h32.csv
2,2.0,0.377005,0,handflapping-20240913-16h32.csv
3,3.0,0.549024,0,handflapping-20240913-16h32.csv
4,4.0,0.334656,0,handflapping-20240913-16h32.csv
